<a href="https://colab.research.google.com/github/Alexandre77777/computer_math/blob/main/10.%20%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0%20%D0%B8%20%D1%80%D0%B0%D0%B7%D0%B2%D0%B5%D1%80%D1%82%D1%8B%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%20%D0%B2%D0%B5%D0%B1-%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D0%BE%D0%B2%20%D0%B4%D0%BB%D1%8F%20%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%20%D0%BA%D0%BE%D0%BC%D0%BF%D1%8C%D1%8E%D1%82%D0%B5%D1%80%D0%BD%D0%BE%D0%B9%20%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B8/%D0%A2%D0%B5%D0%BC%D0%B0_10_%D0%A0%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%B8_%D1%80%D0%B0%D0%B7%D0%B2%D0%B5%D1%80%D1%82%D1%8B%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B2%D0%B5%D0%B1_%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D0%BE%D0%B2_%D0%B4%D0%BB%D1%8F_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87_%D0%BA%D0%BE%D0%BC%D0%BF%D1%8C%D1%8E%D1%82%D0%B5%D1%80%D0%BD%D0%BE%D0%B9_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Тема 10. Разработка и развертывание веб-сервисов для задач компьютерной математики**

## **Введение**


В этом пособии рассмотрим процесс создания и развертывания приложения, где:
- **Бэкенд** разрабатывается с использованием FastAPI и размещается на платформе render.com
- **Фронтенд** разрабатывается на Streamlit
- Приложение выполняет обработку Excel-файлов и генерирует отчеты в формате Markdown

Такая архитектура позволяет разделить логику приложения: бэкенд отвечает за обработку данных, а фронтенд — за пользовательский интерфейс и взаимодействие с пользователем.

## **Раздел №1. Создание бэкенда (main.py)**



Бэкенд приложения будет принимать Excel-файлы, обрабатывать их и возвращать результаты в формате Markdown.

In [ ]:
# main.py
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse
from io import BytesIO
import pandas as pd
import numpy as np
from typing import List
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Инициализация FastAPI приложения
app = FastAPI(
    title="Excel to Markdown API",
    description="API для обработки Excel файлов и генерации отчетов в формате Markdown",
    version="1.0.0"
)

@app.get("/")
def read_root():
    """Корневой маршрут, возвращающий информацию о сервисе"""
    return {
        "message": "Excel to Markdown API работает",
        "endpoints": {
            "/process-excel/": "Загрузка и обработка Excel-файла с генерацией отчета в формате Markdown"
        }
    }

@app.post("/process-excel/")
async def process_excel(file: UploadFile = File(...)):
    """
    Обрабатывает загруженный Excel-файл и возвращает отчет в формате Markdown
    """
    # Проверка формата файла
    if not file.filename.endswith(('.xlsx', '.xls')):
        raise HTTPException(
            status_code=400,
            detail="Поддерживаются только файлы Excel (.xlsx, .xls)"
        )

    try:
        # Чтение содержимого файла
        contents = await file.read()
        buffer = BytesIO(contents)

        # Загрузка Excel-файла в DataFrame
        df = pd.read_excel(buffer)

        # Генерация отчета в формате Markdown на основе данных
        report = generate_markdown_report(df)

        # Создание байтового объекта для хранения отчета
        output = BytesIO(report.encode())
        output.seek(0)

        # Возвращаем отчет как скачиваемый файл
        filename = f"report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
        return StreamingResponse(
            output,
            media_type="text/markdown",
            headers={"Content-Disposition": f"attachment; filename={filename}"}
        )

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Ошибка обработки файла: {str(e)}")

def generate_markdown_report(df: pd.DataFrame) -> str:
    """
    Генерирует отчет в формате Markdown на основе предоставленного DataFrame
    """
    # Создаем заголовок отчета
    report = f"# Отчет по анализу данных\n\n"
    report += f"Дата создания: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"

    # Основная информация о данных
    report += f"## Общая информация\n\n"
    report += f"- **Количество строк**: {df.shape[0]}\n"
    report += f"- **Количество столбцов**: {df.shape[1]}\n"
    report += f"- **Столбцы**: {', '.join(df.columns)}\n\n"

    # Статистика по числовым столбцам
    report += f"## Статистический анализ\n\n"

    # Проверяем наличие числовых столбцов
    numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
    if numeric_columns:
        report += f"### Числовые данные\n\n"
        stats_df = df[numeric_columns].describe().transpose()
        # Форматируем статистику в виде таблицы Markdown
        stats_table = "| Столбец | Количество | Среднее | Ст. отклонение | Мин | 25% | 50% | 75% | Макс |\n"
        stats_table += "| --- | --- | --- | --- | --- | --- | --- | --- | --- |\n"

        for column, row in stats_df.iterrows():
            stats_table += f"| {column} | {row['count']:.0f} | {row['mean']:.2f} | {row['std']:.2f} | {row['min']:.2f} | {row['25%']:.2f} | {row['50%']:.2f} | {row['75%']:.2f} | {row['max']:.2f} |\n"

        report += stats_table + "\n\n"

    # Анализ категориальных данных
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
    if categorical_columns:
        report += f"### Категориальные данные\n\n"

        for column in categorical_columns:
            value_counts = df[column].value_counts().head(5)  # Топ-5 значений
            report += f"#### {column}\n\n"

            # Создаем таблицу с частотами значений
            report += "| Значение | Количество | Процент |\n"
            report += "| --- | --- | --- |\n"

            for value, count in value_counts.items():
                percentage = (count / len(df)) * 100
                report += f"| {value} | {count} | {percentage:.2f}% |\n"

            report += "\n"

    # Анализ пропущенных значений
    report += f"## Анализ пропущенных значений\n\n"
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        report += "| Столбец | Пропущенные значения | Процент пропущенных |\n"
        report += "| --- | --- | --- |\n"

        for column, missing in missing_values.items():
            if missing > 0:
                percentage = (missing / len(df)) * 100
                report += f"| {column} | {missing} | {percentage:.2f}% |\n"

        report += "\n"
    else:
        report += "Пропущенные значения отсутствуют.\n\n"

    # Заключение
    report += "## Выводы\n\n"
    report += "На основе анализа данных можно сделать следующие выводы:\n\n"
    report += "1. Данные содержат информацию о " + str(df.shape[0]) + " записях с " + str(df.shape[1]) + " характеристиками.\n"

    if numeric_columns:
        # Находим столбец с наибольшим средним значением
        max_mean_column = df[numeric_columns].mean().idxmax()
        max_mean_value = df[numeric_columns].mean().max()
        report += f"2. Столбец '{max_mean_column}' имеет наибольшее среднее значение ({max_mean_value:.2f}).\n"

    if missing_values.sum() > 0:
        most_missing = missing_values.idxmax()
        most_missing_count = missing_values.max()
        report += f"3. Столбец '{most_missing}' имеет наибольшее количество пропущенных значений ({most_missing_count}).\n"

    report += "\n"

    return report

**Объяснение main.py:**

**Импорты:**
- FastAPI и связанные модули для создания API.
- BytesIO для работы с данными в памяти.
- pandas для обработки Excel-файлов и данных.
- numpy для числовых вычислений.
- matplotlib и seaborn для возможной визуализации (не используются в текущем коде).
- datetime для добавления временных меток.

**Инициализация FastAPI:**
- Создаем экземпляр приложения с заголовком, описанием и версией.

**Корневой маршрут (@app.get("/")):**
- Возвращает информацию о доступных эндпоинтах API, что полезно для документирования и проверки работоспособности API.

**Эндпоинт обработки Excel-файлов (@app.post("/process-excel/")):**
- Принимает файл через параметр file с типом UploadFile.
- Проверяет расширение файла (.xlsx или .xls).
- Читает содержимое файла, преобразуя его в BytesIO для дальнейшей обработки.
- Загружает данные в DataFrame с помощью pandas.
- Генерирует отчет в формате Markdown с помощью функции generate_markdown_report.
- Возвращает отчет как скачиваемый файл с использованием StreamingResponse.
- Обрабатывает исключения, возвращая соответствующие HTTP-ошибки.

**Функция generate_markdown_report:**
- Принимает DataFrame и возвращает строку с отчетом в формате Markdown.
- Создает заголовок отчета с датой создания.
- Включает общую информацию о данных (количество строк, столбцов, имена столбцов).
- Проводит статистический анализ:
  - Для числовых столбцов: количество, среднее, стандартное отклонение, минимум, квартили, максимум.
  - Для категориальных столбцов: топ-5 значений с процентами.
- Анализирует пропущенные значения по каждому столбцу.
- Формирует выводы на основе анализа данных.

## **Раздел №2. Создание фронтенда на Streamlit (app.py)**



Фронтенд приложения отвечает за предоставление пользовательского интерфейса для загрузки Excel-файлов и получения отчетов.

In [ ]:
# app.py
import streamlit as st
import pandas as pd
import requests
import io
from urllib.parse import urljoin

# Конфигурация приложения
st.set_page_config(
    page_title="Excel Analyzer",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded",
)

# URL бэкенда на render.com
# Замените на свой URL после развертывания на render.com
BACKEND_URL = "https://excel-to-markdown-api.onrender.com"

# Функции для взаимодействия с бэкендом
def check_api_status():
    """Проверяем доступность API"""
    try:
        response = requests.get(BACKEND_URL, timeout=10)
        return response.status_code == 200
    except requests.RequestException:
        return False

def process_excel(file):
    """Отправляем Excel-файл на обработку в API и получаем Markdown-отчет"""
    url = urljoin(BACKEND_URL, "/process-excel/")
    files = {"file": file}
    try:
        response = requests.post(url, files=files)
        if response.status_code == 200:
            return response.content.decode('utf-8')
        else:
            st.error(f"Ошибка при обработке файла: {response.text}")
            return None
    except requests.RequestException as e:
        st.error(f"Ошибка соединения с API: {str(e)}")
        return None

# Интерфейс приложения
def main():
    st.title("📊 Анализатор Excel файлов")
    st.markdown("""
    Этот инструмент позволяет загрузить Excel-файл и получить аналитический отчет
    в формате Markdown. Просто загрузите файл и нажмите кнопку "Анализировать".
    """)

    # Проверка статуса API
    if not check_api_status():
        st.error("⚠️ Не удалось подключиться к API. Пожалуйста, проверьте соединение или попробуйте позже.")
        return

    # Загрузка файла
    uploaded_file = st.file_uploader("Выберите Excel файл", type=['xlsx', 'xls'])

    if uploaded_file is not None:
        # Показываем предварительный просмотр данных
        try:
            df = pd.read_excel(uploaded_file)
            st.subheader("Предварительный просмотр данных")
            st.dataframe(df.head(5))

            # Получение основных статистик для информации
            st.subheader("Базовая информация")
            col1, col2, col3 = st.columns(3)
            col1.metric("Строки", df.shape[0])
            col2.metric("Столбцы", df.shape[1])
            col3.metric("Пропущенные значения", df.isna().sum().sum())

            # Сбрасываем указатель файла для повторного чтения
            uploaded_file.seek(0)

            if st.button("Анализировать"):
                with st.spinner("Обрабатываем данные..."):
                    markdown_report = process_excel(uploaded_file)

                if markdown_report:
                    st.success("Отчет успешно создан!")

                    # Показываем отчет в интерфейсе
                    st.subheader("Отчет")
                    st.markdown(markdown_report)

                    # Предоставляем возможность скачать отчет
                    st.download_button(
                        label="Скачать отчет",
                        data=markdown_report,
                        file_name="report.md",
                        mime="text/markdown",
                    )

        except Exception as e:
            st.error(f"Ошибка при чтении файла: {str(e)}")

if __name__ == "__main__":
    main()

**Объяснение app.py:**

**Импорты:**
- Streamlit для создания интерактивного веб-интерфейса.
- pandas для предварительного просмотра данных.
- requests для HTTP-запросов к API.
- io для работы с байтовыми потоками.
- urljoin для корректного формирования URL.

**Конфигурация приложения:**
- st.set_page_config задает заголовок, иконку, широкую раскладку и расширенное начальное состояние боковой панели.

**URL бэкенда:**
- BACKEND_URL: Важно! Замените "https://excel-to-markdown-api.onrender.com" на реальный URL вашего API после его развертывания на Render.

**Функции для взаимодействия с бэкендом:**
- check_api_status: Проверяет доступность API, отправляя GET-запрос и проверяя статус ответа.
- process_excel: Отправляет файл на обработку в API с помощью POST-запроса и возвращает полученный отчет в формате Markdown или None в случае ошибки.

**Функция main:**
- Создает заголовок и описание приложения.
- Проверяет доступность API; если API недоступен, выводит сообщение об ошибке.
- Предоставляет компонент для загрузки файла с ограничением по типам (только Excel-файлы).
- При загрузке файла:
  - Показывает предварительный просмотр данных с помощью st.dataframe.
  - Отображает базовую информацию о данных (строки, столбцы, пропущенные значения).
  - Сбрасывает указатель файла для повторного чтения с помощью uploaded_file.seek(0).
  - Предоставляет кнопку "Анализировать" для отправки файла в API.
  - При нажатии на кнопку:
    - Показывает спиннер во время обработки.
    - Получает отчет с помощью функции process_excel.
    - Если отчет получен успешно, отображает его и предоставляет кнопку для скачивания.
- Обрабатывает исключения, возникающие при чтении файла.

**Условие __name__ == "__main__":**
- Обеспечивает выполнение функции main только при прямом запуске скрипта, что важно при развертывании.

## **Раздел №3. Развертывание приложения**

### **1. Развертывание бэкенда на render.com**



1. **Подготовка файла requirements.txt**

   ```
fastapi==0.109.2
uvicorn==0.27.1
pandas==2.2.0
numpy==1.26.3
openpyxl==3.1.2
xlrd==2.0.1
matplotlib==3.8.2
seaborn==0.13.1
python-multipart==0.0.9
streamlit==1.30.0
requests==2.31.0
   ```

2. **Шаги по развертыванию на render.com**:
   - Зарегистрируйтесь на [render.com](https://render.com)
   - Создайте новый Web Service, указав репозиторий с вашим кодом
   - В настройках укажите:
     - Команда запуска: `uvicorn main:app --host 0.0.0.0 --port $PORT`
   - Нажмите "Create Web Service"
   - После успешного развертывания, получите URL вашего API (например, https://excel-to-markdown-api.onrender.com)

### **2. Развертывание фронтенда на Streamlit**


. **Шаги по развертыванию на Streamlit Cloud**:
   - Зарегистрируйтесь на [Streamlit Cloud](https://streamlit.io/cloud)
   - Создайте новый проект, указав репозиторий с вашим кодом
   - Укажите путь к главному файлу: app.py
   - Не забудьте изменить URL бэкенда в app.py на URL, полученный после развертывания на render.com
   - Нажмите "Deploy"

## **Раздел 3. Скрипт для генерации тестового Excel файла**



Ниже представлен Python-скрипт для создания тестового Excel-файла, который можно использовать для проверки работы разработанного приложения:

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Установка seed для воспроизводимости результатов
np.random.seed(42)
random.seed(42)

# Определяем количество строк
num_rows = 100

# Создаем словарь для хранения данных
data = {}

# Числовые данные
data['Продажи'] = np.random.normal(1000, 200, num_rows).round(2)
data['Маржа'] = np.random.uniform(0.05, 0.25, num_rows).round(4)
data['Количество'] = np.random.randint(1, 100, num_rows)
data['Цена'] = (data['Продажи'] / data['Количество']).round(2)

# Дата
start_date = datetime(2023, 1, 1)
data['Дата'] = [start_date + timedelta(days=random.randint(0, 365)) for _ in range(num_rows)]

# Категориальные данные
categories = ['Электроника', 'Одежда', 'Книги', 'Товары для дома', 'Спорт', 'Продукты']
data['Категория'] = [random.choice(categories) for _ in range(num_rows)]

customer_types = ['Розница', 'Опт', 'Онлайн']
data['ТипКлиента'] = [random.choice(customer_types) for _ in range(num_rows)]

regions = ['Север', 'Юг', 'Восток', 'Запад', 'Центр']
data['Регион'] = [random.choice(regions) for _ in range(num_rows)]

# Добавляем числовой столбец с пропущенными значениями
data['Рейтинг'] = np.random.uniform(1, 5, num_rows).round(1)
# Добавляем пропущенные значения (примерно 10%)
missing_indices = np.random.choice(num_rows, size=int(num_rows * 0.1), replace=False)
for idx in missing_indices:
    data['Рейтинг'][idx] = np.nan

# Добавляем категориальный столбец с пропущенными значениями
data['СпособОплаты'] = [random.choice(['Карта', 'Наличные', 'Банковский перевод', 'PayPal']) for _ in range(num_rows)]
# Добавляем пропущенные значения (примерно 15%)
missing_indices = np.random.choice(num_rows, size=int(num_rows * 0.15), replace=False)
for idx in missing_indices:
    data['СпособОплаты'][idx] = None

# Добавляем бинарные данные
data['Возврат'] = [random.choice([0, 1]) for _ in range(num_rows)]
data['СрочныйЗаказ'] = [random.choice([0, 1]) for _ in range(num_rows)]

# Создаем DataFrame из словаря
df = pd.DataFrame(data)

# Сохраняем как Excel файл
df.to_excel('test_data.xlsx', index=False)

print("Тестовый Excel-файл 'test_data.xlsx' успешно создан.")

Тестовый Excel-файл 'test_data.xlsx' успешно создан.


### **Пояснение:**




1. **Импорт библиотек**:
   - `pandas` для работы с данными и сохранения в Excel
   - `numpy` для генерации случайных чисел
   - `random` для выбора категориальных значений
   - `datetime` для создания дат

2. **Настройка генератора случайных чисел**:
   - Установка seed для воспроизводимости результатов при каждом запуске скрипта

3. **Создание структуры данных**:
   - Скрипт создает датасет, имитирующий данные о продажах, с различными типами данных
   - **Числовые данные**: Продажи, Маржа, Количество, Цена
   - **Временные данные**: Дата
   - **Категориальные данные**: Категория товара, Тип клиента, Регион
   - **Данные с пропусками**: Рейтинг (числовой), Способ оплаты (категориальный)
   - **Бинарные данные**: Возврат (0/1), Срочный заказ (0/1)

4. **Генерация пропущенных значений**:
   - В столбце "Рейтинг" случайно удалено 10% значений
   - В столбце "СпособОплаты" случайно удалено 15% значений

5. **Сохранение результата**:
   - Данные сохраняются в файл "test_data.xlsx" в текущей директории

После генерации файла его можно загрузить в Ваше приложение для получения аналитического отчета в формате Markdown.

## **Заключение**



Представленный подход к созданию приложения с разделением на бэкенд и фронтенд имеет ряд преимуществ:

1. **Масштабируемость**: бэкенд можно усилить для обработки больших файлов без воздействия на фронтенд
2. **Разделение ответственности**: бэкенд отвечает за обработку данных, фронтенд - за интерфейс
3. **Безопасность**: логика обработки данных скрыта на сервере
4. **Гибкость**: можно создавать дополнительные фронтенды для одного бэкенда
